<a href="https://colab.research.google.com/github/Exabyte-io/api-examples/blob/dev/examples/workflow/get_workflows.ipynb" target="_parent">
<img alt="Open in Google Colab" src="https://user-images.githubusercontent.com/20477508/128780728-491fea90-9b23-495f-a091-11681150db37.jpeg" width="150" border="0">
</a>

# Overview

Inside this example we contact [Workflow](https://docs.mat3ra.com/api/Workflows/get_workflows) endpoint to obtain Band Gap workflow and adjust settings.

# Complete Authorization Form and Initialize Settings

This will also determine environment and set all environment variables. We determine if we are using Jupyter Notebooks or Google Colab to run this tutorial.

If you are running this notebook from Google Colab, Colab takes ~1 min to execute the following cell.

ACCOUNT_ID and AUTH_TOKEN - Authentication parameters needed for when making requests to [Mat3ra.com's API Endpoints](https://docs.mat3ra.com/rest-api/endpoints/).

MATERIALS_PROJECT_API_KEY - Authentication parameter needed for when making requests to [Material Project's API](https://materialsproject.org/open)

ORGANIZATION_ID - Authentication parameter needed for when working with collaborative accounts https://docs.mat3ra.com/collaboration/organizations/overview/

> <span style="color: orange">**NOTE**</span>: If you are running this notebook from Jupyter, the variables ACCOUNT_ID, AUTH_TOKEN, MATERIALS_PROJECT_API_KEY, and ORGANIZATION_ID should be set in the file [settings.json](../../utils/settings.json) if you need to use these variables. To obtain API token parameters, please see the following link to the documentation explaining how to get them: https://docs.mat3ra.com/accounts/ui/preferences/api/

In [ ]:
# @title Authorization Form
ACCOUNT_ID = "ACCOUNT_ID"  # @param {type:"string"}
AUTH_TOKEN = "AUTH_TOKEN"  # @param {type:"string"}
MATERIALS_PROJECT_API_KEY = "MATERIALS_PROJECT_API_KEY"  # @param {type:"string"}
ORGANIZATION_ID = "ORGANIZATION_ID"  # @param {type:"string"}

import os
import sys

if "COLAB_JUPYTER_IP" in os.environ:
    os.environ.update(
        dict(
            ACCOUNT_ID=ACCOUNT_ID,
            AUTH_TOKEN=AUTH_TOKEN,
            MATERIALS_PROJECT_API_KEY=MATERIALS_PROJECT_API_KEY,
            ORGANIZATION_ID=ORGANIZATION_ID,
        )
    )

    !GIT_BRANCH="dev"; export GIT_BRANCH; curl -s "https://raw.githubusercontent.com/Exabyte-io/api-examples/${GIT_BRANCH}/scripts/env.sh" | bash

if sys.platform == "emscripten":
    apiConfig = data_from_host.get("apiConfig")
    os.environ.update(data_from_host.get("environ", {}))
    os.environ.update(
        dict(
            ACCOUNT_ID=apiConfig.get("accountId"),
            AUTH_TOKEN=apiConfig.get("authToken"),
            MATERIALS_PROJECT_API_KEY=MATERIALS_PROJECT_API_KEY,
            ORGANIZATION_ID=apiConfig.get("organizationId") or "",
        )
    )

    import micropip

    await micropip.install("mat3ra-api-examples", deps=False)
    await micropip.install("mat3ra-utils")
    from mat3ra.utils.jupyterlite.packages import install_packages

    await install_packages("api_examples")

## Imports

In [ ]:
from utils.settings import ENDPOINT_ARGS, ACCOUNT_ID
from utils.generic import display_JSON

from exabyte_api_client.endpoints.workflows import WorkflowEndpoints

## Set Parameters

- **QUERY**: A query describing the documents to find. See [Meteor collection](https://docs.meteor.com/api/collections.html#Mongo-Collection-find) for more information. 

- **limit**: Maximum number of results to return. See [Meteor collection](https://docs.meteor.com/api/collections.html#Mongo-Collection-find) for more information.

In [ ]:
OWNER_ID = os.getenv("ORGANIZATION_ID") or ACCOUNT_ID
# If this doesn't exist, we need to set model to be with LDA
QUERY = {"name": {"$regex": "Band Gap.*LDA"}, "owner._id": OWNER_ID}

OPTIONS = {"limit": 2}

## Initialize the endpoint

Initialize a helper class to interact with `WorkflowEndpoints`. This only has to be done once.

In [ ]:
endpoint = WorkflowEndpoints(*ENDPOINT_ARGS)

## List workflows

Contact the endpoint to list workflows according to the query above.

In [ ]:
workflows = endpoint.list(QUERY, OPTIONS)

## Print workflows

Print the list of workflows saved under the corresponding variable in pretty JSON below.

In [ ]:
display_JSON(workflows)

In [ ]:
wf = workflows[0]

In [ ]:
from exabyte_api_client.endpoints.materials import MaterialEndpoints
from exabyte_api_client.endpoints.jobs import JobEndpoints

material_endpoints = MaterialEndpoints(*ENDPOINT_ARGS)
OWNER_ID = os.getenv("ORGANIZATION_ID") or ACCOUNT_ID
print(ENDPOINT_ARGS)

# We will upload the material (WS2 bilayer) first, then get its ID and use it in the job configuration.
materials = material_endpoints.list({"isDefault": True, "owner._id": OWNER_ID})[0:5]

In [ ]:
material_id = materials[0]["_id"]
material_id

In [ ]:
job_endpoints = JobEndpoints(*ENDPOINT_ARGS)
config = {
    "owner": {"_id": OWNER_ID},
    "_material": {"_id": material_id},
    "workflow": wf,
    "name": "TEST JOB BG - from Python",
}
# print(config["workflow"])

In [ ]:
# Create job:
job = job_endpoints.create(config)
# job_endpoints.submit(job["_id"])

### Modify the job to set kgrid parameters

In [ ]:
modifier = {
    "workflow.subworkflows.0.units.0.context": {
        "kgridExtraData": {"materialHash": "a665723ef7429caef6ca89385fe25bae"},
        "kgrid": {
            "dimensions": [3, 3, 1],
            "shifts": [0, 0, 0],
            "reciprocalVectorRatios": [1, 1, 1],
            "gridMetricType": "KPPRA",
            "gridMetricValue": 2,
            "preferGridMetric": False,
        },
        "isKgridEdited": True,
        "subworkflowContext": {},
    }
}

In [ ]:
# Update job with the modifier
job_endpoints.update(id_=job["_id"], modifier=modifier)